# Bài toán

> **Thu thập dữ liệu báo Việt Nam (Dantri - BTVN)**

> Hoàn thành bằng cách điền vào phần #### CODE theo hướng dẫn

    ```
    #### CODE
    ####
    ```

Cũng giống như VNExpress - Demo


Mục tiêu:

- Hiểu được quy trình thu thập dữ liệu từ trang báo của Việt Nam.
- Thu thập dữ liệu (bài báo) từ các trang báo của Việt Nam để làm dữ liệu cho các bước xử lý sau.

Đầu ra:

-  Tập file JSON chứa các bài bài báo có các trường dữ liệu:

    - `url`: link dẫn đến bài báo
    - `title`: tiêu đề bài báo
    - `description`: tóm tắt bài báo
    - `content`: nội dung bài báo
    - `metadata`: trường dữ liệu bổ sung

        - `cat`: thể loại bài báo
        - `subcat`: thể loại con của bài báo
        - `published_date`: thời gian xuất bản
        - `author`: người viết
    
- Ví dụ về một bài báo:

    ```
    {
        "url": "https://dantri.com.vn/an-sinh/khong-nen-giam-che-do-om-dau-...",
        "title": "Không nên giảm chế độ ốm đau của người lao động",
        "description": "Chủ tịch công đoàn công ty Intel Product đề nghị không nên ...",
        "content": "[Giảm quyền lợi chế độ ốm đau ảnh hưởng ...",
        "metadata": {
            "cat": "AN SINH",
            "subcat": "DÂN SINH",
            "published_date": 1716159600,
            "author": "Tùng Nguyên"
        },
    },
    ```

# Các bước tiến hành

## Chuẩn bị các thư viện cần thiết

In [1]:
# Suitable for Google Colab, for local please follow the external instructions and ignore this line
# and follows https://docs.google.com/document/d/14jK9d6KHJYX0b-gFAVqAghUxT7OLAM0nP2IovL7_Rjs/edit?usp=sharing
# !apt install -qq chromium-chromedriver

In [2]:
# Install selenium
%pip install -qq selenium

# Tạo thư mục để chứa data
!mkdir data

Note: you may need to restart the kernel to use updated packages.


A subdirectory or file data already exists.


In [3]:
# selenium import
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException

# other imports
import os
import json

In [4]:
# selenium setups
## https://www.tutorialspoint.com/selenium/selenium_webdriver_chrome_webdriver_options.htm

chrome_options = webdriver.ChromeOptions()

chrome_options.add_argument('--headless') # must options for Google Colab
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument("--disable-extensions")
chrome_options.add_argument("--disable-gpu")


In [5]:
MAGAZINE_NAME = "dantri"
HOME_PAGE = "https://dantri.com.vn/"

## Thu thập dữ liệu

> **Các bước thu thập bài báo**


1. News categories: Thu thập tất cả các thể loại báo của website
2. News urls: Thu thập một số đường dẫn dựa vài từng thể loại báo của website đó
3. News articles: Thu thập và xử lý từng bài báo dựa vào đường dẫn của bước trước

### Thu thập thể loại bài báo của website: Crawling categories

> **Các bước thu thập**

1. Vào trang chủ của báo
2. Thu thập các thể loại ở mục menu


In [6]:
driver = webdriver.Chrome(options=chrome_options)
# Vào trang web chính, mặc định phải chờ toàn bộ trang webload mới xong
driver.get(HOME_PAGE)

In [7]:
# Choose the menu #### CODE
driver.find_element(by=By.CLASS_NAME, value="menu-more").click()

In [8]:
# Lấy hết tất cả thể loại từ đây #### CODE
cats = []
cat_items = driver.find_elements(By.CSS_SELECTOR, ".nf-menu > li > a")
cats = [{'cat_name': item.text, 'url': item.get_attribute("href")} for item in cat_items]
cats, len(cats)

([{'cat_name': 'XÃ HỘI', 'url': 'https://dantri.com.vn/xa-hoi.htm'},
  {'cat_name': 'THẾ GIỚI', 'url': 'https://dantri.com.vn/the-gioi.htm'},
  {'cat_name': 'KINH DOANH', 'url': 'https://dantri.com.vn/kinh-doanh.htm'},
  {'cat_name': 'BẤT ĐỘNG SẢN',
   'url': 'https://dantri.com.vn/bat-dong-san.htm'},
  {'cat_name': 'THỂ THAO', 'url': 'https://dantri.com.vn/the-thao.htm'},
  {'cat_name': 'VIỆC LÀM',
   'url': 'https://dantri.com.vn/lao-dong-viec-lam.htm'},
  {'cat_name': 'NHÂN ÁI', 'url': 'https://dantri.com.vn/tam-long-nhan-ai.htm'},
  {'cat_name': 'SỨC KHỎE', 'url': 'https://dantri.com.vn/suc-khoe.htm'},
  {'cat_name': 'GIẢI TRÍ', 'url': 'https://dantri.com.vn/giai-tri.htm'},
  {'cat_name': 'XE ++', 'url': 'https://dantri.com.vn/o-to-xe-may.htm'},
  {'cat_name': 'SỨC MẠNH SỐ', 'url': 'https://dantri.com.vn/suc-manh-so.htm'},
  {'cat_name': 'GIÁO DỤC', 'url': 'https://dantri.com.vn/giao-duc.htm'},
  {'cat_name': 'AN SINH', 'url': 'https://dantri.com.vn/an-sinh.htm'},
  {'cat_name': 'P

In [9]:
driver.close()

### Thu thập một số đường dẫn dựa vài từng thể loại báo của website đó: News urls


> **Cách thu thập**

Từ những thể loại và đường link tương ứng, ta lần lượt vào từng thể loại và lấy đường dẫn của các bài báo trong mục đó của trang web đó.



#### Cài đặt tham số

In [10]:
# Đặt số lượng đường dẫn cần lấy trong mỗi thể loại báo
NUM_ARTICLES_PER_CAT = 10 # có thể tăng lên
DATA_URL_FILE = "data/dantri_url.json"
EXCLUDING_CATEGORIES = ["NHÂN ÁI"] # Exclude
chrome_options.page_load_strategy = "normal"
driver = webdriver.Chrome(options=chrome_options)

#### Chạy thử nghiệm

In [27]:
driver.get(cats[0]['url'])


In [28]:
titles = driver.find_elements(By.CSS_SELECTOR, ".article-title > a")
titles[0].get_attribute("href")

'https://dantri.com.vn/xa-hoi/chuyen-vu-huy-giay-khai-sinh-cua-hoc-sinh-lop-12-toi-chu-tich-ha-noi-20240807155023239.htm'

#### Chạy thật

In [142]:
NUM_ARTICLES_PER_CAT = 400

In [143]:
all_cats_url = set()
def crawl_url_for_category(driver: webdriver, page_url: str):
    cat_urls = set()
    while len(cat_urls) < NUM_ARTICLES_PER_CAT:
        driver.get(page_url)
        titles = driver.find_elements(By.CSS_SELECTOR, ".article-title > a")
        for title in titles:
            try:
                url = title.get_attribute("href").strip()
                if url.startswith(HOME_PAGE) and url not in all_cats_url:
                    all_cats_url.add(url)
                    cat_urls.add(url)
            except StaleElementReferenceException:
                print("Check driver reference, consider re-run notebook")
            except NoSuchElementException:
                print("Check code logic and edge cases !! (No Such Element)")
        page_url = driver.find_element(by=By.CSS_SELECTOR, value="a.page-item.next").get_attribute("href")
    return cat_urls

In [144]:
urls_by_cat = {}
for (i,cat) in enumerate(cats):
    # if i < 9: continue
    cat_name = cat["cat_name"]
    url = cat['url']
    if cat_name not in EXCLUDING_CATEGORIES:
        print(f"You are at {cat}.")
        urls = crawl_url_for_category(driver, url)
        urls_by_cat[cat_name] = list(urls)

You are at {'cat_name': 'XÃ HỘI', 'url': 'https://dantri.com.vn/xa-hoi.htm'}.
You are at {'cat_name': 'THẾ GIỚI', 'url': 'https://dantri.com.vn/the-gioi.htm'}.
You are at {'cat_name': 'KINH DOANH', 'url': 'https://dantri.com.vn/kinh-doanh.htm'}.
You are at {'cat_name': 'BẤT ĐỘNG SẢN', 'url': 'https://dantri.com.vn/bat-dong-san.htm'}.
You are at {'cat_name': 'THỂ THAO', 'url': 'https://dantri.com.vn/the-thao.htm'}.
You are at {'cat_name': 'VIỆC LÀM', 'url': 'https://dantri.com.vn/lao-dong-viec-lam.htm'}.
You are at {'cat_name': 'SỨC KHỎE', 'url': 'https://dantri.com.vn/suc-khoe.htm'}.
You are at {'cat_name': 'GIẢI TRÍ', 'url': 'https://dantri.com.vn/giai-tri.htm'}.
You are at {'cat_name': 'XE ++', 'url': 'https://dantri.com.vn/o-to-xe-may.htm'}.
You are at {'cat_name': 'SỨC MẠNH SỐ', 'url': 'https://dantri.com.vn/suc-manh-so.htm'}.
You are at {'cat_name': 'GIÁO DỤC', 'url': 'https://dantri.com.vn/giao-duc.htm'}.
You are at {'cat_name': 'AN SINH', 'url': 'https://dantri.com.vn/an-sinh.ht

In [145]:
if len(all_cats_url) < NUM_ARTICLES_PER_CAT * (len(cats) - len(EXCLUDING_CATEGORIES)):
    print("BE CAREFUL NOT TO OVERWRITE ALREADY-EXISTENT DATA")
else:
    with open(DATA_URL_FILE, "w", encoding="utf-8") as fOut:
        json.dump(urls_by_cat, fOut, ensure_ascii=False, indent=4)
    driver.close()
    len(all_cats_url)

### Thu thập và xử lý từng bài báo dựa vào đường dẫn của bước trước: News articles


> **Cách thu thập**

Từ đường dẫn ở trong phần trước, ta lần lượt vào từng đường link đó và thu thập thông tin về bài báo.

#### Cài đặt thông số

In [12]:
# Filepath cho cái trước
FILE_URL_PATH = "data/dantri_url.json"
MAX_ARTICLES_PER_CAT = 400 # Nếu set = None thì là tất cả các urls ở file trước
DATA_FOLDER_OUTPUT = "data/dantri"
!mkdir "$DATA_FOLDER_OUTPUT"
chrome_options.page_load_strategy = "eager"

A subdirectory or file data/dantri already exists.


In [13]:
# Đọc file url
with open(FILE_URL_PATH, "r", encoding='utf-8') as fIn: url_data = json.load(fIn)
len(url_data)

17

#### Chạy thử nghiệm

In [150]:
driver = webdriver.Chrome(options=chrome_options)

# Một số url để thử nghiệm
SAMPLE_ARTICLE_URLS = [
    "https://dantri.com.vn/xa-hoi/hang-van-du-khach-o-lai-tren-cac-dao-quang-ninh-hai-phong-cho-bao-tan-20240722185725192.htm",
    "https://dantri.com.vn/xa-hoi/nhieu-du-khach-ngoi-thung-o-to-ban-tai-luu-thong-tren-duong-pho-sam-son-20240722163805286.htm",
    "https://dantri.com.vn/the-gioi/ong-trump-doi-boi-thuong-sau-khi-tong-thong-biden-dung-tranh-cu-20240722215254170.htm",
    "https://dantri.com.vn/lao-dong-viec-lam/lao-dong-viet-ve-chan-dung-bac-ho-tong-bi-thu-nguyen-phu-trong-bang-2-tay-20240721101224503.htm",
    "https://dantri.com.vn/khoa-hoc-cong-nghe/chu-cho-lac-nha-dung-dam-mua-nhieu-ngay-cho-chu-den-don-20240722153845731.htm",
    "https://dantri.com.vn/giai-tri/muon-kiep-nhan-sinh-cua-gs-john-vu-nguyen-phong-sap-phat-hanh-20200430134253084.htm" # Author
    # "https://dantri.com.vn/xa-hoi/nguoi-dan-tphcm-xep-hang-thap-huong-tuong-nho-tong-bi-thu-nguyen-phu-trong-20240722193718325.htm" # we ignore this one (emagazine), maybe extensions in the future if you want handle indepedently,
    # "https://dantri.com.vn/xa-hoi/toan-van-bai-viet-khoi-day-niem-tu-hao-ve-dang-cua-tong-bi-thu-20240131111512837.htm" # similar reason like above
]

# chọn url
SAMPLE_ARTICLE_URL = SAMPLE_ARTICLE_URLS[-1]

In [151]:
driver.get(SAMPLE_ARTICLE_URL)

In [14]:
# We ignore emagazine for now ..
def check_emagazine(driver: webdriver):
    is_emagazine = True
    try:
        driver.find_element(by=By.CLASS_NAME, value="emagazine")
    except NoSuchElementException:
        is_emagazine = False
    return is_emagazine

check_emagazine(driver)

False

Function for extracting article's content and metadata

In [15]:
# Tìm kiếm title #### CODE
def title_of(driver: webdriver): return driver.find_element(By.CLASS_NAME, "title-page").text

# Tìm kiếm description #### CODE
def description_of(driver: webdriver):
    description = driver.find_element(By.CLASS_NAME, "singular-sapo").text
    DANTRI_PHRASE = "(Dân trí) - "
    if DANTRI_PHRASE in description:
        description = description.replace(DANTRI_PHRASE, "")
    return description

# Thu thập thể loại #### CODE
def categories_of(driver: webdriver):
    breadcrumb = driver.find_element(By.CSS_SELECTOR, "main > div > ul")
    breadcrumb = breadcrumb.find_elements(By.TAG_NAME, "a")
    main_cat = breadcrumb[0].get_attribute("title")
    sub_cat = breadcrumb[1].get_attribute("title") if len(breadcrumb) > 1 else ""
    return (main_cat, sub_cat)

# Thu thập ngày #### CODE
def date_of(driver: webdriver): return driver.find_element(By.CSS_SELECTOR, "time.author-time").get_attribute("datetime")

# Thu thập tác giả
def author_of(driver: webdriver):
    author = "Unknown"
    try:
        author = driver.find_element(By.CSS_SELECTOR, ".author-name b").text
    except: pass
    return author

# Tìm kiếm contents
def contents_of(driver: webdriver):
    article = driver.find_element(By.CLASS_NAME, "singular-content")
    children = article.find_elements(By.XPATH, "./*")
    contents = []
    for idx, child in enumerate(children):
        text = child.text.strip()
        if child.tag_name in ["p", "h1", "h2", "h3", "h4"]: contents.append(text)
        elif child.tag_name == "figure":
            if len(text): contents.append(f"[{text}]" if len(text) <= 100 else text)
    return contents

In [129]:
driver.get("https://dantri.com.vn/giai-tri/phim-di-giua-troi-ruc-ro-tranh-cai-ve-trang-phuc-dan-toc-e-kip-noi-gi-20240807124810718.htm")

In [132]:
title_of(driver), date_of(driver), categories_of(driver), author_of(driver), contents_of(driver)

('Phim "Đi giữa trời rực rỡ" tranh cãi về trang phục dân tộc, ê-kíp nói gì?',
 '2024-08-07 14:20',
 ('Giải trí', 'Điện ảnh'),
 'Lạc Thành',
 ['"Người Dao không mặc lễ phục khi đi chăn trâu"',
  'Bộ phim Đi giữa trời rực rỡ lên sóng từ 31/7, khắc họa cuộc sống của Pu (Thu Hà Ceri) - cô gái 18 tuổi người Dao đỏ -  trên con đường trưởng thành.',
  'Trong phim, người được ghép đôi với Pu là Chải (Long Vũ) - thanh niên giàu nhất bản. Chải yêu Pu say đắm và tìm mọi cách để cưới cô. Điều này cũng gây không ít cản trở đối với Pu trên con đường học vấn.',
  'Diễn viên Long Vũ (trái) và Thu Hà (phải) ở hậu trường phim "Đi giữa trời rực rỡ" (Ảnh: Facebook nhân vật).',
  'Bên cạnh những lời khen về cảnh quay hùng vĩ và nên thơ của thiên nhiên vùng cao kết hợp với những nét văn hóa truyền thống độc đáo của đồng bào dân tộc Dao, khán giả nhận thấy trang phục và cách sử dụng trang phục, tập quán của người Dao đỏ có một số chỗ chưa phù hợp với thực tế.',
  '"Người Dao không mặc bộ lễ phục đính chùm bô

In [14]:
driver.close()

#### Chạy thật

In [16]:

class MagazineArticleException(Exception): pass

def get_content_metadata(driver, article_url):

    """
    Extracts and returns metadata and content from a given article URL.

    :param driver: Selenium WebDriver instance.
    :param article_url: URL of the article to extract data from.
    :return: Dictionary containing article metadata and content.
    """

    driver.get(article_url)
    if check_emagazine(driver): raise MagazineArticleException("We ignore MagazineArticle")

    main_cat, sub_cat = categories_of(driver)
    return {
        "url": article_url,
        "title": title_of(driver),
        "description": description_of(driver),
        "content": "\n".join(contents_of(driver)),
        "metadata": {
            "cat": main_cat,
            "subcat": sub_cat,
            "published_date": date_of(driver),
            "author": author_of(driver)
        }
    }

In [17]:
idx = 0
for cat, urls in url_data.items():
    idx += 1
    if idx < 14: continue
    print(f"Thu thập dữ liệu thể loại {cat} ..")

Thu thập dữ liệu thể loại DU LỊCH ..
Thu thập dữ liệu thể loại ĐỜI SỐNG ..
Thu thập dữ liệu thể loại TÌNH YÊU ..
Thu thập dữ liệu thể loại KHOA HỌC - CÔNG NGHỆ ..


In [21]:
driver = webdriver.Chrome(options=chrome_options)
# driver.set_page_load_timeout(24)  # Increase timeout
idx = 0
for cat, urls in url_data.items():
    idx += 1
    if idx < 14: continue
    print(f"Thu thập dữ liệu thể loại {cat} ..")
    count_crawled = 0
    cat_data = []
    for url in urls:
        try:
            cat_data.append(get_content_metadata(driver, url))
            count_crawled += 1
            if MAX_ARTICLES_PER_CAT and count_crawled >= MAX_ARTICLES_PER_CAT:
                break
        # NoSuchElementException
        except (StaleElementReferenceException) as e:
            print(f"Bug at url: {url}, with StaleElementReferenceException")
            driver.refresh()
            continue
        except (NoSuchElementException) as e:
            print(f"Bug at url: {url}, with NoSuchElementException")
            driver.refresh()
            continue
        except MagazineArticleException as e:
            continue

    name_file_cat = cat.lower().replace(" ", "-") + ".json"
    with open(os.path.join(DATA_FOLDER_OUTPUT, name_file_cat), "w", encoding='utf-8') as fOut:
        json.dump(cat_data, fOut, ensure_ascii=False, indent=4)

driver.close()

Thu thập dữ liệu thể loại DU LỊCH ..
Thu thập dữ liệu thể loại ĐỜI SỐNG ..
Thu thập dữ liệu thể loại TÌNH YÊU ..
Bug at url: https://dantri.com.vn/tinh-yeu-gioi-tinh/sau-tieng-go-cua-dem-tan-hon-toi-soc-khi-biet-su-that-ve-chong-20240430101019797.htm, with NoSuchElementException
Thu thập dữ liệu thể loại KHOA HỌC - CÔNG NGHỆ ..


In [136]:
# Xem 1 sample
cat_data[-1]

{'url': 'https://dantri.com.vn/khoa-hoc-cong-nghe/trung-quoc-doi-pho-thien-tai-the-nao-20240805073552285.htm',
 'title': 'Trung Quốc đối phó thiên tai thế nào?',
 'description': 'Trung Quốc đang tăng cường sử dụng công nghệ vệ tinh và radar trong công tác cứu trợ thiên tai khi các hiện tượng thời tiết cực đoan ngày càng thường xuyên hơn.',
 'content': '[Hình ảnh vệ tinh có thể được sử dụng để phối hợp cho các nỗ lực cứu hộ do thiên tai (Ảnh: CNS/AFP).]\nTrung Quốc đang đối mặt với hàng loạt trận mưa lũ khắc nghiệt bậc nhất trong lịch sử kể từ khi hứng chịu siêu bão Gaemi hồi cuối tháng 7.\nTheo Tân Hoa Xã, 87 con sông ở 7 tỉnh (gồm Hắc Long Giang, Cát Lâm, Liêu Ninh, Chiết Giang, Phúc Kiến, Giang Tây, Quảng Đông) và Khu tự trị Nội Mông ở miền bắc Trung Quốc đã chứng kiến mực nước dâng cao.\nMưa lớn ở tỉnh Hồ Nam, miền trung Trung Quốc cũng gây ra 3 vụ vỡ đê trong vòng chưa đầy 20 giờ và buộc hàng nghìn người dân phải sơ tán khi nước lũ vượt qua ngưỡng kỷ lục.\nĐối phó tình trạng thiên 

## Lưu dữ liệu

Nếu bạn chạy ở máy cá nhân thì không cần, nhưng nếu mà chạy ở Colab thì nên lưu dữ liệu vào trong Google Drive


In [ ]:
# For Google Colab
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Set to your folder
FOLDER_SAVED_GOOGLE_COLAB = "/content/drive/MyDrive/crawl-news/"

# Copy
!cp -r data $FOLDER_SAVED_GOOGLE_COLAB